# Project 1 - Finding Lane lines on the Road

In this Project I have used the concepts leanred in Lesson 1 of Self Driving Car nano-degree of Udacity.This code is written to draw lane lines on the road from an already existing recorded video. Series of steps are followed to reach the goal of this project.

## Step : 1
Import all the neceessary libraries 

In [1]:
import matplotlib.pyplot as plt # Importing pyplot functions from Matplotlib to edit the plots
import matplotlib.image as mpimg# Importing image function from Matplotlib to read the images
import numpy as np # For performing operaions on image 
import cv2 # For using Open CV libraries
from moviepy.editor import VideoFileClip # to deal with video editing
from IPython.display import HTML # to output the video
import math 
%matplotlib inline

## Step : 2
Creating all the Important Functions

In [2]:
def grayscale(img):   
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
 
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Step 3
Creating the function which will take an Image frame from the video and draw lines on in

In [3]:
old_r=1
old_l=1
old_right=0
old_left=0
old_intercept_r=0
old_intercept_l=0


In [6]:
def process_image(image):
    
    gray=grayscale(image) # Converting it to grayscale
      
    kernel_size = 3   
    blur_gray=gaussian_blur(gray,kernel_size) # applying Guassian Blur on the Image  
    
    low_threshold = 50
    high_threshold =155
    edges=canny(blur_gray,low_threshold,high_threshold)  # Using edge detecction algorithms
    
    
    imshape = image.shape    
    vertices = np.array([[(170,480),(430,400), (600,400), (860,480)]], dtype=np.int32)
    masked_edges = region_of_interest(edges,vertices)  
    
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20  # maximum gap in pixels between connectable line segments
    #threshold = 50     # minimum number of votes (intersections in Hough grid cell)
    #min_line_length = 100 #minimum number of pixels making up a line
    #max_line_gap = 160  # maximum gap in pixels between connectable line segments
    #line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    #lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
     #                       min_line_length, max_line_gap)
    lines = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
   
    
    # identifying the left lane lines and the right lines by calulating the slope
    r=0
    right=0
    l=0
    left=0
    intercept_r=0
    intercept_l=0
    for line in lines:
        for x1,y1,x2,y2 in line:
            if (((y2-y1)/(x2-x1)) > 1) | (((y2-y1)/(x2-x1)) <-1) | ((x2-x1)==0):  # rejecting some of the outliers
                a=1
            elif ((y2-y1)/(x2-x1))>0.5:   # good range to select the right lane lines
                right=right+((y2-y1)/(x2-x1))
                r=r+1
                intercept_r=intercept_r+y2-((y2-y1)/(x2-x1))*x2
            elif ((y2-y1)/(x2-x1)) < -0.3:    # good range to select the left lane lines
                left=left+((y2-y1)/(x2-x1))
                l=l+1
                intercept_l=intercept_l+y2-((y2-y1)/(x2-x1))*x2
             
   
    if (r==0): 
        r=old_r
        intercept_r=old_intercept_r       
        right=old_right
        
        
    if (l==0):
        l=old_l
        intercept_l=old_intercept_l
        left=old_left
   
    p=0.2
    q=0.8
    mean_right=p*(right/r)+q*(old_right/old_r)
    mean_left=p*(left/l)+q*(old_left/old_l)
    mean_right_intercept=p*(intercept_r/r)+q*(old_intercept_r/old_r)
    mean_left_intercept=p*(intercept_l/l)+q*(old_intercept_l/old_l)
    
        
    
    Y1=650
    Y2=330
    X1=int((Y1-mean_left_intercept)/mean_left)
    X2=int((Y2-mean_left_intercept)/mean_left)


    Y11=650
    Y22=330
    X11=int((Y11-mean_right_intercept)/mean_right)
    X22=int((Y22-mean_right_intercept)/mean_right)
    lines = np.copy(image)*0
    cv2.line(lines,(X1,Y1),(X2,Y2),(255,0,0),10)
    cv2.line(lines,(X11,Y11),(X22,Y22),(255,0,0),10)
    image_weighted=cv2.addWeighted(image,0.8,lines,2.0,0)
    
    global old_r 
    old_r = r
    global old_l
    old_l = l
    global old_right 
    old_right = right
    global old_left
    old_left = left
    global old_intercept_r
    old_intercept_r = intercept_r
    global old_intercept_l 
    old_intercept_l = intercept_l
    
    return image_weighted

In [7]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
##clip1 = VideoFileClip("test_videos/challenge.mp4")
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:06<00:00, 36.01it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 



In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [9]:
white_output = 'test_videos_output/solidYellowLeft.mp4'

clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:19<00:00, 35.29it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 



In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))